In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk, Frame
import mysql.connector
import cv2
import numpy as np
import os
import time

# Initialize Main Window
window = tk.Tk()
window.title("Advanced Face Recognition System")
window.geometry("800x600")
window.configure(bg='#2c2f33')

notebook = ttk.Notebook(window)
notebook.pack(pady=10, expand=True)

attendance_frame = Frame(notebook, width=800, height=500, bg='#23272a')
train_frame = Frame(notebook, width=800, height=500, bg='#23272a')

attendance_frame.pack(fill='both', expand=True)
train_frame.pack(fill='both', expand=True)

notebook.add(attendance_frame, text='Attendance System')
notebook.add(train_frame, text='Generate & Train Dataset')

style = ttk.Style()
style.configure('TButton', font=('Helvetica', 14), padding=10)
style.configure('TLabel', background='#2c2f33', foreground='white', font=('Helvetica', 16))

# Database Connection Function
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="attendance_system"
    )

# Attendance Functions
def mark_attendance(user_id):
    db = connect_db()
    cursor = db.cursor()
    cursor.execute("INSERT INTO attendance (user_id, status) VALUES (%s, %s)", (user_id, 'present'))
    db.commit()
    cursor.close()
    db.close()

def get_user_name(user_id):
    db = connect_db()
    cursor = db.cursor()
    cursor.execute("SELECT name FROM my_table WHERE id = %s", (user_id,))
    result = cursor.fetchone()
    cursor.close()
    db.close()
    return result[0] if result else "Unknown"

# Face Detection and Attendance
def detect_faces():
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        messagebox.showerror("Error", "Camera not accessible.")
        return

    recognized_ids = set()
    confidence_threshold = 70

    while True:
        ret, frame = cap.read()
        if not ret:
            messagebox.showerror("Error", "Failed to read from camera.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            id, confidence = clf.predict(gray[y:y + h, x:x + w])
            if confidence < confidence_threshold:
                user_name = get_user_name(id)
                if id not in recognized_ids:
                    mark_attendance(id)
                    recognized_ids.add(id)
                label = f"{user_name} ({confidence:.2f}%)"
                color = (0, 255, 0)
            else:
                label = "UNKNOWN"
                color = (0, 0, 255)

            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

        cv2.imshow("Face Detection - Press Enter to Stop", frame)
        if cv2.waitKey(1) == 13:  # Enter key
            break

    cap.release()
    cv2.destroyAllWindows()

# Dataset Generation
def generate_dataset():
    if not all([name_var.get(), age_var.get(), address_var.get()]):
        messagebox.showinfo('Error', 'All fields are required!')
        return

    db = connect_db()
    cursor = db.cursor()
    cursor.execute("SELECT MAX(id) FROM my_table")
    user_id = (cursor.fetchone()[0] or 0) + 1

    cursor.execute("INSERT INTO my_table (id, name, age, address) VALUES (%s, %s, %s, %s)",
                   (user_id, name_var.get(), age_var.get(), address_var.get()))
    db.commit()

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Error", "Camera not accessible.")
        return

    img_id = 0
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    while img_id < 100:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)

        for (x, y, w, h) in faces:
            img_id += 1
            cv2.imwrite(f"data/user.{user_id}.{img_id}.jpg", gray[y:y + h, x:x + w])
            progress_bar['value'] = (img_id / 100) * 100
            window.update_idletasks()

        cv2.imshow("Dataset Generation - Press Enter to Stop", frame)
        if cv2.waitKey(1) == 13:  # Enter key
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Result', 'Dataset Generation Completed')

# Train Classifier
def train_classifier():
    faces, ids = [], []
    for image in os.listdir("data"):
        img = cv2.imread(f"data/{image}", cv2.IMREAD_GRAYSCALE)
        faces.append(img)
        ids.append(int(image.split('.')[1]))

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, np.array(ids))
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training Completed')

# User Input Fields
name_var = tk.StringVar()
age_var = tk.StringVar()
address_var = tk.StringVar()

form_frame = Frame(train_frame, bg='#23272a')
form_frame.pack(pady=10)

ttk.Label(form_frame, text="Name:").grid(row=0, column=0, padx=10, pady=10)
ttk.Entry(form_frame, textvariable=name_var, width=30).grid(row=0, column=1, padx=10, pady=10)
ttk.Label(form_frame, text="Age:").grid(row=1, column=0, padx=10, pady=10)
ttk.Entry(form_frame, textvariable=age_var, width=30).grid(row=1, column=1, padx=10, pady=10)
ttk.Label(form_frame, text="Address:").grid(row=2, column=0, padx=10, pady=10)
ttk.Entry(form_frame, textvariable=address_var, width=30).grid(row=2, column=1, padx=10, pady=10)

progress_bar = ttk.Progressbar(train_frame, length=400, mode='determinate')
progress_bar.pack(pady=20)

ttk.Button(train_frame, text="Generate Dataset", command=generate_dataset).pack(pady=10)
ttk.Button(train_frame, text="Train Classifier", command=train_classifier).pack(pady=10)
ttk.Button(attendance_frame, text="Detect Faces", command=detect_faces).pack(pady=20)

window.mainloop()


In [3]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import mysql.connector
import pandas as pd
from tkcalendar import DateEntry  # Ensure tkcalendar is installed: pip install tkcalendar

# Database connection
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="attendance_system"
    )

# Fetch filtered attendance records from the database
def fetch_attendance(start_date=None, end_date=None):
    db = connect_db()
    cursor = db.cursor()
    query = """SELECT my_table.name, attendance.status, attendance.timestamp
               FROM attendance
               JOIN my_table ON attendance.user_id = my_table.id"""

    # Apply date filter if provided
    if start_date and end_date:
        query += " WHERE DATE(attendance.timestamp) BETWEEN %s AND %s"
        cursor.execute(query, (start_date, end_date))
    else:
        cursor.execute(query)

    records = cursor.fetchall()
    cursor.close()
    db.close()
    return records

# Populate TreeView with attendance data
def populate_table(tree, start_date=None, end_date=None):
    for row in tree.get_children():
        tree.delete(row)  # Clear previous data

    records = fetch_attendance(start_date, end_date)
    for record in records:
        tree.insert('', tk.END, values=record)

# Export attendance data to CSV
def export_csv(start_date=None, end_date=None):
    records = fetch_attendance(start_date, end_date)
    if not records:
        messagebox.showinfo("Export", "No attendance data to export.")
        return

    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])
    if file_path:
        df = pd.DataFrame(records, columns=["Name", "Status", "Timestamp"])
        df.to_csv(file_path, index=False)
        messagebox.showinfo("Export", "Attendance exported successfully!")

# Admin Dashboard Interface
def admin_dashboard():
    dashboard = tk.Toplevel()
    dashboard.title("Admin Dashboard")
    dashboard.geometry("900x500")
    dashboard.configure(bg='#1e1e1e')

    # Title
    tk.Label(dashboard, text="Attendance Records", font=("Helvetica", 24, "bold"), 
             bg='#1e1e1e', fg='white').pack(pady=10)

    # Date Filter Frame
    filter_frame = tk.Frame(dashboard, bg='#1e1e1e')
    filter_frame.pack(pady=5)

    tk.Label(filter_frame, text="Start Date:", font=("Helvetica", 14), bg='#1e1e1e', fg='white').grid(row=0, column=0, padx=5)
    start_date = DateEntry(filter_frame, width=15, background='darkblue', foreground='white', date_pattern='yyyy-mm-dd')
    start_date.grid(row=0, column=1, padx=5)

    tk.Label(filter_frame, text="End Date:", font=("Helvetica", 14), bg='#1e1e1e', fg='white').grid(row=0, column=2, padx=5)
    end_date = DateEntry(filter_frame, width=15, background='darkblue', foreground='white', date_pattern='yyyy-mm-dd')
    end_date.grid(row=0, column=3, padx=5)

    ttk.Button(filter_frame, text="Filter", 
               command=lambda: populate_table(tree, start_date.get_date(), end_date.get_date())).grid(row=0, column=4, padx=10)

    # Table Frame with Scrollbars
    table_frame = tk.Frame(dashboard)
    table_frame.pack(pady=10, fill=tk.BOTH, expand=True)

    tree = ttk.Treeview(table_frame, columns=("Name", "Status", "Timestamp"), show='headings', height=15)
    tree.heading("Name", text="Name")
    tree.heading("Status", text="Status")
    tree.heading("Timestamp", text="Timestamp")

    for col in ("Name", "Status", "Timestamp"):
        tree.column(col, anchor=tk.CENTER, width=200)

    # Scrollbars
    vsb = ttk.Scrollbar(table_frame, orient="vertical", command=tree.yview)
    hsb = ttk.Scrollbar(table_frame, orient="horizontal", command=tree.xview)
    tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

    vsb.pack(side=tk.RIGHT, fill=tk.Y)
    hsb.pack(side=tk.BOTTOM, fill=tk.X)
    tree.pack(fill=tk.BOTH, expand=True)

    # Populate initial data
    populate_table(tree)

    # Export Button
    ttk.Button(dashboard, text="Export to CSV", 
               command=lambda: export_csv(start_date.get_date(), end_date.get_date())).pack(pady=10)

# Main Tkinter Window
root = tk.Tk()
root.title("Face Recognition Attendance System")
root.geometry("400x300")
root.configure(bg='#2c2f33')

# Main Window Title
tk.Label(root, text="Attendance System", font=("Helvetica", 24, "bold"), 
         bg='#2c2f33', fg='white').pack(pady=20)

# Button to Open Admin Dashboard
ttk.Button(root, text="Open Admin Dashboard", command=admin_dashboard).pack(pady=10)

# Run the Application
root.mainloop()
